<font size="5">
 <div class="alert alert-block alert-info"><b>Master in Data Science - Iscte <b>
     </div>
</font> 
 
 
     
    
  <font size="5"> OEOD </font>
  
  
  
  <font size="3"> **Diana Aldea Mendes**, November 2023 </font>
  
   
  <font size="3"> *diana.mendes@iscte-iul.pt* </font> 
  
    
 
  
    
  <font color='blue'><font size="5"> <b>Week 8 - Case study 3 - RL for Healthcare <b></font></font>

# RL for Healthcare


- Reinforcement Learning applications in healthcare
    - Drug dosage (Q-learning, deep Q-learning) (for example insulin for diabetes)
    - Dynamic Treatment Recommendation (chronic diseases and critical care) (actor-critic with RNN)
    - Medical Diagnosis
    - Resource scheduling and Allocation
    - Drug discovery
    - Health Management

- Take a look here: https://healthgym.ai/
- https://vadim.me/publications/heartpole/


- Reinforcement learning for healthcare applications:

1. Define the Environment:
   - Identify the healthcare problem you want to address, such as treatment plan optimization or clinical decision-making.
   - Define the state representation: Determine the relevant patient information, medical history, diagnostic test results, and other relevant factors that will constitute the state space.
   - Define the action space: Determine the set of possible actions, such as treatment options, medication dosages, or diagnostic tests to be ordered.
   - Define the reward function: Design a reward function that reflects the effectiveness or quality of the treatment plan or decision made. For example, the reward could be based on patient health outcomes, cost-effectiveness, or adherence to medical guidelines.

2. Implement the Reinforcement Learning Algorithm:
   - Choose a reinforcement learning algorithm suitable for your healthcare problem, such as Q-learning, deep Q-networks (DQN), or actor-critic methods.
   - Initialize the necessary components, such as the Q-table or the neural network architecture.
   - Implement the training loop:
     - Sample a state from the environment.
     - Select an action using an exploration-exploitation strategy, such as epsilon-greedy or softmax.
     - Execute the chosen action and observe the next state and the reward.
     - Update the Q-values or adjust the model weights based on the chosen algorithm.
     - Repeat the above steps until convergence or a sufficient number of iterations.

3. Data Collection and Preprocessing:
   - Collect and preprocess the healthcare data required for training and evaluation. This may include patient records, medical imaging data, laboratory results, or clinical guidelines.
   - Apply necessary data transformations and feature engineering techniques to prepare the data for input to the reinforcement learning algorithm.

4. Training and Evaluation:
   - Split the data into training and evaluation sets.
   - Train the reinforcement learning model on the training data using the implemented algorithm.
   - Evaluate the trained model's performance on the evaluation data, measuring relevant metrics such as patient outcomes, cost-effectiveness, or guideline adherence.

5. Iterative Improvement:
   - Analyze the results and iteratively refine the reinforcement learning model or the environment based on the observed performance.
   - Adjust hyperparameters, modify the reward function, or consider incorporating additional data sources to enhance the model's effectiveness and generalizability.





# Example 1 - Hospital Warehouse

- **Goal**: develop a model that uses RL for the case of a hospital warehouse.

## Model Description:

- **Agent**: 
    - A single hospital warehouse subject to a stochastic customer demand (such as, for example, a Uniform distribution between values a and b).
    - The warehouse is the **decision making agent**, deciding if it will either order additional supplies or if it will deliver supplies to customers.
- **Reward**:
    - The Goal is to maximize a reward, which for a hospital is the internal revenue obtained from a delivery, this is, income minus expenses.

- The Reward experienced by this Warehouse is the total revenue, which consists of
    - a. Sales Price of the items sold
    - b. MINUS the holding cost of the Inventory
    - c. MINUS the penality for Customer Demand not met
- **Actions**
    - The warehouse holds inventory (no maximum, and held in whole quantities) and can take two actions:
        - a. Sell inventory to a customer
        - b. Receive inventory from its supplier
    - The warehouse can only do one of this actions at a time.
- **State**: The State of the model is the warehouse inventory level

In [ ]:
# import libraries

import numpy as np      
import math
import time             # calculate execution time
import pandas as pd     # manipulate Dataframes 

np.set_printoptions(precision=2, suppress=True)  # number of digits for float type and do not write 1e-n

# high-quality figures
%config InlineBackend.figure_format = 'svg'


___________________________________

- **Model parameters**:
    - The model has cost and price parameters defined at the beginning of the model (do not substantially change when the model is executed).
    - The Warehouse has as many *states* as the number of possible units in its stock
    - The number of units (contained by the warehouse) is between 0 and **max_state** (that is, the Warehouse *size*). 
    - The initial state (initial warehouse stock) will be a random number between 0 and max_state.

In [ ]:
cost_inventory = 2     # The cost of holding inventory
purchase_price = 20    # The price at which the inventory is bought
sales_price = 50       # The price at which the inventory is sold

max_state = 800
initial_state = np.random.randint(max_state)

______________________________________

- *Reward structure matrix* (**R matrix**) is a square matrix of size (max_state * max_state). 
- It contains the rewards and therefore also defines the possible actions that can be taken. 
- First we initialize the R Matrix, and after we fill the reward values 
- The X and Y coordinates of the Matrix correspond to the initial and final values of the decision.
- For example, the R Matrix value at [10, 45] represents that the state is moving from 10 to 45, this means that 35 stock items have been added to the warehouse. 
- Therefore, the reward is negative and correspond to the cost of purchasing 35 new stock items + the cost of inventory at the beginning of the period.

In [ ]:
# initialize R-matrix (all entries are zero)

R = np.matrix(np.zeros([max_state,max_state]),dtype=int)

# define the rule to fill the reward values in the R matrix

for y in range(0, max_state):
    for x in range(0, max_state):
        R[x,y] = np.maximum((x-y)*sales_price,0)-np.maximum((y-x)*purchase_price,0)-x*cost_inventory
        
print('R: \n', R)

___________________________________

- The **Q matrix** capture the total future reward for an agent from a given state after a certain action
- The Q matrix has the same dimensions as the R Matrix. 
- It is initialized with random numbers entries. 
- The *learning rate* and the *discount rate* are also required parameters.

In [ ]:
## initialize Q matrix with random number entries

Q = np.matrix(np.random.random([max_state,max_state]))

# define hyperparameters

learning_rate = 0.5
discount = 0.7
EPISODES =1000
STEPS = 200
PRINT_EVERY = EPISODES/50
Pepsilon_init = 0.8   # initial value for the decayed-epsilon-greedy method
Pepsilon_end = 0.1    # final value for the decayed-epsilon-greedy method

___________________________________

- Now we define the **functions** we are going to use in the algorithm. 
- First, we consider a function that determine the **available actions** from which we can choose the next action.


In [ ]:
## function that determine the set of possible actions for the next action

def available_actions(state, customers):
    # The available actions are 
    #    a.- Meeting a customer requirement (going to s: state-order)
    #    b.- Buying Inventory from Supplier (going to s: state+purchase)
    purchase = np.arange(state, max_state) # Calculate all possible future states due to purchases from the current state
    # print('Purchase: ',purchase)
    new_customers_state =[]
    new_customers_state = [np.maximum(state-x,0) for x in customers] # calculate the possible states from customers in the current state
    # print('new_customers_state: ', new_customers_state)
    return np.concatenate((purchase,new_customers_state))

________________________________________

- Next, we define a function that randomly chooses which action to be performed within the range of available actions (that is, the future action). 
- There is a range of options given by:
    1. Always choose the action that has the **maximum Q value** for the current_state (*exploit*)
    2. Always choose a random action from the available actions for the current_state (*explore*)
    3. Alternate between exploitation and exploration with a certain probability (which may change overtime)
- The general recommendation is to start exploring most of the time and gradually decrease exploration to maximize exploitation.
- For this we use the decayed-epsilon-greedy method where epsilon is the probability that a random action is chosen.

In [ ]:
def sample_next_action(available_act, epsilon):
    # choose the type of the next action to take. 
    # 1 for a random next action with probability epsilon 
    # 0 for a greedy next action with probability (1-epsilon)
    random_action = np.random.binomial(n=1, p=epsilon, size=1)
    
    if random_action == 1:
        # This is the option for full exploration - always random
        # print('random action')
        next_action = int(np.random.choice(available_act, 1))
    else:
        # This is the option for full exploitation - always use what we know (Greedy method)
        # Choose the next actions from the available actions, and the one with the highest Q Value
        # print('greedy action')
        next_action = int(np.where(Q[current_state,] == np.max(Q[current_state,available_act]))[1])
    
    # now calculate the amount that is being sold or purchsed, if at all
    if next_action < current_state:
        Qsale = current_state-next_action
        Qpurchase = 0
    else: 
        Qpurchase = next_action - current_state
        Qsale = 0
    return next_action, Qsale, Qpurchase
    
def cost_inventory_backlog(current_state):
    if current_state<=0:
        return cost_backlog
    else:
        return cost_inventory

__________________________________________


- The following function updates the **Q table** with a formula (Bellman Q-equation) that requires the parameters:
    - Q[current_state, action] = value to update (the action is the future state value the agent decided to take).
    - learning_rate = value between 0 and 1 indicating how much new information over ride sold information.
    - R[current_state, action] = Reward obtained when transitioning from current_state to future_state 
    - Q[future_state, a] = the max Q value of the possible actions a in the future_state



In [ ]:
# this function updates the Q matrix/table according to the path selected and the Q learning algorithm

def update(current_state, action):
   
    max_index = np.where(Q[action,] == np.max(Q[action,]))[1] # index for the maximum Q value in the future_state
    # print('Q[action,]: \n', Q[action,])
    # print('Current State: ', current_state)
    # print('Action: ', action)
    # print('Max Index:', max_index)
    
    # just in case there are more than one maxima, in which case we choose one randomly
    if max_index.shape[0] > 1:
        max_index = int(np.random.choice(max_index, size=1))
    else:
        max_index = int(max_index)
    max_value = Q[action, max_index] # this is the maximum Q value in the future state given the action that generates that maximum value
    
    # Q learning formula
    Q[current_state, action] = (1-learning_rate)*Q[current_state, action] + learning_rate*(R[current_state, action] + discount*max_value)

In [ ]:
### training the simulation
# Training

start_time = time.time()
epsilon = Pepsilon_init
epsilon_delta = (Pepsilon_init - Pepsilon_end)/EPISODES

calculation_times = []
total_reward = []
total_demand =[]

total_jump = []
jump_max = []
jump_min = []
jump_av =[]
jump_sd = []

total_state= []
state_max = []
state_min = []
state_av =[]
state_sd = []

current_state = 0

for episode in range(EPISODES):
    # Initialize values for Step
    total_reward_episode = 0
    start_time_episode = time.time()
    
    state_episode = []
    jump_episode = []

    # determine if this episode is generating a status output
    if episode%PRINT_EVERY == 0:
        print('Calc. Episode {} of {}, {:.1f}% progress'.format(episode, EPISODES, 100*(episode/EPISODES)))
    # Execute the steps in the Episode    
    for step in range(STEPS):
        # Create a customer for this step
        customers = []
        customers.append(np.random.randint(0,max_state))
        # Calculate the actions (future states) that are available from current state
        available_act = available_actions(current_state, customers)
        # Choose an action from the available future states
        action = sample_next_action(available_act, epsilon)
        # Update the Q table 
        update(current_state, action[0])

        # record the states for the step
        
        total_state.append(current_state)
        state_episode.append(current_state)
        total_demand.append(customers[0])
        total_reward_episode += R[current_state, action[0]]
        total_jump.append(action[0] - current_state)
        jump_episode.append(action[0] - current_state)
        # update the state for the next step
        current_state = action[0]

    # record the states for the Episode
    total_reward.append(total_reward_episode) # Total reward for the episode
    calculation_times.append(time.time()-start_time_episode)
    
    jump_max.append(np.max(jump_episode))
    jump_min.append(np.min(jump_episode))
    jump_av.append(np.mean(jump_episode))
    jump_sd.append(np.std(jump_episode))
    
    state_max.append(np.max(state_episode))
    state_min.append(np.min(state_episode))
    state_av.append(np.mean(state_episode))
    state_sd.append(np.std(state_episode))

    # Update parameters for the next episode
    epsilon = Pepsilon_init - episode*epsilon_delta
    current_state = np.random.randint(0, int(Q.shape[0]))

# print out the total calculation time
print('total calculation time: {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
## plot results

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
%matplotlib inline

## subplot with (2x3)=6 figures

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(8, 6))

MA_total_reward = pd.DataFrame(total_reward)

Rolling_total_reward = MA_total_reward.rolling(window=5).mean()

## labels and other info for each one of the 6 subplots

axs[0,0].plot(MA_total_reward, label='Episode')
axs[0,0].plot(Rolling_total_reward, color='r', label ='MA(5)')
axs[0,0].set_title('Total Rewards')
axs[0,0].set_ylabel('Rewards [$]')
axs[0,0].set_xlabel('Episodes')
axs[0,0].grid(axis='y', alpha=0.75)
axs[0,0].grid(axis='x', alpha=0.75)

axs[1,0].plot(calculation_times)
axs[1,0].set_title('Calc. Times')
axs[1,0].set_xlabel('Episodes')
axs[1,0].set_ylabel('Calculation times [s]')
axs[1,0].grid(axis='y', alpha=0.75)
axs[1,0].grid(axis='x', alpha=0.75)

axs[0,1].hist(total_state,color='#0504aa',alpha=0.7, rwidth=0.85)
axs[0,1].set_title('States Histogram')
axs[0,1].set_xlabel('State')
axs[0,1].set_ylabel('Frequency')
axs[0,1].set_xlim(xmin=0, xmax=max_state)
axs[0,1].grid(axis='y', alpha=0.75)

axs[1,1].plot(jump_max,color='b', label = 'max')
axs[1,1].plot(jump_min,color='r', label = 'min')
axs[1,1].plot(jump_av,color='g', label = 'av')
axs[1,1].plot(jump_sd,color='y', label = 'sd')
axs[1,1].set_title('Jumps')
axs[1,1].legend()
axs[1,1].set_xlabel('Episode')
axs[1,1].set_ylabel('Jump Value')
axs[1,1].grid(axis='y', alpha=0.75)

axs[0,2].hist(total_jump,color='#0504aa',alpha=0.7, rwidth=0.85)
axs[0,2].set_title('Jump Histogram')
axs[0,2].set_xlabel('New_State-Old_State')
axs[0,2].set_ylabel('Frequency')
axs[0,2].set_xlim(xmin=-max_state, xmax=max_state)
axs[0,2].grid(axis='y', alpha=0.75)

axs[1,2].plot(state_max,color='b', label = 'max')
axs[1,2].plot(state_min,color='r', label = 'min')
axs[1,2].plot(state_av,color='g', label = 'av')
axs[1,2].plot(state_sd,color='y', label = 'sd')
axs[1,2].set_title('States')
axs[1,2].legend()
axs[1,2].set_xlabel('Episode')
axs[1,2].set_ylabel('State Value')
axs[1,2].grid(axis='y', alpha=0.75)


plt.tight_layout()
plt.show()

In [ ]:
MA_total_reward = pd.DataFrame(total_reward)

Rolling_total_reward = MA_total_reward.rolling(window=50).mean()

plt.figure(figsize=(8,6))
plt.plot(MA_total_reward, label='Episode')
plt.plot(Rolling_total_reward, color='r', label ='MA(50)')
plt.title('Total Rewards')
plt.ylabel('Rewards [$]')
plt.xlabel('Episodes')
plt.legend()
plt.show()

# Example 2 - Reinforcement Learning for Effective Clinical Trials

- Multi-armed bandits (MAB) is a Reinforcement Learning (RL) problem that has wide applications and is gaining popularity.
- Multi-armed bandits extend RL by ignoring the state and try to balance between exploration and exploitation.
- Website design and clinical trials are some areas where MAB algorithms are frequently applied.
- Contextual bandits takes MAB further by adding an element of Context from the problem domain.
- Contextual bandits can improve effectiveness of MAB problems but also be applied to new areas like Recommender systems.

https://www.infoq.com/articles/multi-armed-bandits-reinforcement-learning/

- At first step, will be created a simple simulated multi-armed bandit problem simulator class. 
- It will define a 2-armed bandit with each arm pull generating a reward of either 0 or 1. 
- We will design the simulation such that arm1 will give us maximum rewards. 
- We will draw 1000 samples by drawing each arm and plot the distribution of rewards in a figure below.

In [ ]:
import numpy as np
 
# simulate the bandit problem
class MABandit_Problem:
  def __init__(self):
    # we build two bandits
    self.samples = {}
    self.num_samples = 1000
    # create samples for both arms
    self.samples[0] = np.random.normal(0, 0.1, self.num_samples)
    self.samples[0] = self.samples[0] < 0.1
    self.samples[1] = np.random.normal(0.2, 0.3, self.num_samples)
    self.samples[1] = self.samples[1] < 0.1
    self.pointer = 0
  # method for acting on the bandits - return reward
  def pull_arm(self, k):
    reward = 0
    # arms should be 0 or 1
    if k<0 or k>1:
      return -1
    # get the reward
    reward = self.samples[k][self.pointer]
    # update pointer
    self.pointer += 1
    # rotate the pointer
    if self.pointer >= self.num_samples:
      self.pointer = 0
    return reward

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

mab = MABandit_Problem() 
rewards1 = []
rewards2 = []
# sample the two arms
for _ in range(100):
  rewards1.append(mab.pull_arm(0))
  rewards2.append(mab.pull_arm(1))
# plot the rewards for each arm
# reward is 0 or 1
plt.rcParams["figure.figsize"] = (10, 8)
fig, ax = plt.subplots(2)
line1, = ax[0].plot(rewards1, linestyle='None', marker='o')
ax[0].set_title('Rewards distribution for Arm 1')
line2, = ax[1].plot(rewards2, linestyle='None', marker='o')
ax[1].set_title('Rewards distribution for Arm 2')
plt.show()

In [ ]:
import random
 
# Classic A/B testing - Pure exploration
class AB_Testing:
  # initialize bandit class
  def __init__(self):
    self.mab = MABandit_Problem()
    self.total_runs = 1000
    self.arm_flag = False
    self.arm_rewards = [0, 0]
    self.arm_counts = [0, 0]
    self.arm_rew_rate = [0, 0]
  # method for acting on the bandits - return reward
  def run_test(self):
    # for each experiment
    for _ in range(self.total_runs):  
      # pull a random arm
      arm = int(self.arm_flag)
      self.arm_flag = True if not self.arm_flag else False
      # calculate reward
      self.arm_rewards[arm] += self.mab.pull_arm(arm)
      self.arm_counts[arm] += 1
    # calculate total rate of reward
    self.arm_rew_rate = np.divide(self.arm_rewards, self.arm_counts)
    print("Number of each arm pulls = ", self.arm_counts)
    print("Rate of reward for each arm = ", self.arm_rew_rate)
    print('-'*100)
    print("Finding: Arm 1 gives maximum reward - we only end up running %d runs through Arm 2"%(self.arm_counts[1]))
# run the test
test = AB_Testing()
test.run_test()
 


In [ ]:
import math
import random
 
# Epsilon greedy - explore vs exploit trade-off
class EPS_Greedy_Testing:
  def __init__(self):
    self.mab = MABandit_Problem()
    self.total_runs = 1000
    self.epsilon = 0.2
    self.arm_rewards = [0, 0]
    self.arm_counts = [0, 0]
    self.arm_rew_rate = [0, 0]
  # method for acting on the bandits - return reward
  def run_test(self):
    # for each experiment
    for _ in range(self.total_runs):
      arm = -1
      # pull arm following epsilon greedy policy
      if random.random() < self.epsilon:
        # exploration - randomly select arm
        arm = random.randrange(2)
      else:
        # exploitation - choose max reward rate arm
        arm = np.argmax(self.arm_rew_rate)
      # add reward
      self.arm_rewards[arm] += self.mab.pull_arm(arm)
      self.arm_counts[arm] += 1      
      # calculate rate of reward
      self.arm_rew_rate = np.divide(self.arm_rewards, self.arm_counts)
    print("Number of each arm pulls = ", self.arm_counts)
    print("Rate of reward for each arm = ", self.arm_rew_rate)
    print('-'*100)
    print("Finding: Arm 1 gives maximum reward - we only end up running %d runs through Arm 2"%(self.arm_counts[1]))
 
test = EPS_Greedy_Testing()
test.run_test()
 


# Example 3 - Diabetes simulator

## A Type-1 Diabetes simulator implemented in Python for Reinforcement Learning purpose

- **simglucose** library
- This simulator is a python implementation of the FDA-approved [UVa/Padova Simulator (2008 version)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4454102/) for research purpose only. 
- The simulator includes 30 virtual patients, 10 adolescents, 10 adults, 10 children.

- Jinyu Xie. Simglucose v0.2.1 (2018). Available: https://github.com/jxx123/simglucose. 


- **simglucose** supports gymnasium! 


### Main Features

- Simulation environment follows [OpenAI gym](https://github.com/openai/gym) and [rllab](https://github.com/rll/rllab) APIs. 
- It returns observation, reward, done, info at each step, which means the simulator is "reinforcement-learning-ready".
- Supports customized reward function. 
- The reward function is a function of blood glucose measurements in the last hour. 
- By default, the reward at each step is `risk[t-1] - risk[t]`. `risk[t]` is the risk index at time `t` defined in this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2903980/pdf/dia.2008.0138.pdf).
- Supports parallel computing. The simulator simulates multiple patients in parallel using [pathos multiprocessing package](https://github.com/uqfoundation/pathos) (you are free to turn parallel off by setting `parallel=False`).
- The simulator provides a random scenario generator (`from simglucose.simulation.scenario_gen import RandomScenario`) and a customized scenario generator (`from simglucose.simulation.scenario import CustomScenario`). 
- **Command line user-interface will guide you through the scenario settings**.
- The simulator provides the most basic basal-bolus controller for now. 
- It provides very simple syntax to implement your own controller, like Model Predictive Control, PID control, reinforcement learning control, etc.
- You can specify random seed in case you want to repeat your experiments.
- The simulator will generate several plots for performance analysis after simulation. 
- The plots include blood glucose trace plot, Control Variability Grid Analysis (CVGA) plot, statistics plot of blood glucose in different zones, risk indices statistics plot.


In [ ]:
## Installation
#!pip install simglucose

In [ ]:
## Quick Start

### Use simglucose as a simulator and test controllers

# Run the simulator user interface

from simglucose.simulation.user_interface import simulate
simulate()

In [ ]:
## You are free to implement your own controller, and test it in the simulator. For example,


from simglucose.simulation.user_interface import simulate
from simglucose.controller.base import Controller, Action


class MyController(Controller):
    def __init__(self, init_state):
        self.init_state = init_state
        self.state = init_state

    def policy(self, observation, reward, done, **info):
        '''
        Every controller must have this implementation!
        ----
        Inputs:
        observation - a namedtuple defined in simglucose.simulation.env. For
                      now, it only has one entry: blood glucose level measured
                      by CGM sensor.
        reward      - current reward returned by environment
        done        - True, game over. False, game continues
        info        - additional information as key word arguments,
                      simglucose.simulation.env.T1DSimEnv returns patient_name
                      and sample_time
        ----
        Output:
        action - a namedtuple defined at the beginning of this file. The
                 controller action contains two entries: basal, bolus
        '''
        self.state = observation
        action = Action(basal=0, bolus=0)
        return action

    def reset(self):
        '''
        Reset the controller state to inital state, must be implemented
        '''
        self.state = self.init_state


ctrller = MyController(0)
simulate(controller=ctrller)


In [ ]:
# you can specify a lot more simulation parameters through `simulation`:

#simulate(sim_time=my_sim_time,
#         scenario=my_scenario,
#        controller=my_controller,
#        start_time=my_start_time,
#        save_path=my_save_path,
#        animate=False,
#        parallel=True)

In [ ]:
### OpenAI Gym usage

# Using default reward

import gym

# Register gym environment. By specifying kwargs,
# you are able to choose which patient or patients to simulate.
# patient_name must be 'adolescent#001' to 'adolescent#010',
# or 'adult#001' to 'adult#010', or 'child#001' to 'child#010'
# It can also be a list of patient names
# You can also specify a custom scenario or a list of custom scenarios
# If you chose a list of patient names or a list of custom scenarios,
# every time the environment is reset, a random patient and scenario will be
# chosen from the list

from gym.envs.registration import register
from simglucose.simulation.scenario import CustomScenario
from datetime import datetime


In [ ]:
start_time = datetime(2023, 11, 19, 0, 0, 0)
meal_scenario = CustomScenario(start_time=start_time, scenario=[(1,50)])

#register(
#    id='simglucose-adolescent2-v0',
#    entry_point='simglucose.envs:T1DSimEnv',
#    kwargs={'patient_name': 'adolescent#002',
#           'custom_scenario': meal_scenario}
#)

In [ ]:
env = gym.make('simglucose-adolescent2-v0')

In [ ]:
observation = env.reset()
for t in range(100):
    env.render(mode='human')
    print(observation)
    # Action in the gym environment is a scalar
    # representing the basal insulin, which differs from
    # the regular controller action outside the gym
    # environment (a tuple (basal, bolus)).
    # In the perfect situation, the agent should be able
    # to control the glucose only through basal instead
    # of asking patient to take bolus
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t + 1))
        break

In [ ]:
# Customized reward function

import gym
from gym.envs.registration import register


def custom_reward(BG_last_hour):
    if BG_last_hour[-1] > 180:
        return -1
    elif BG_last_hour[-1] < 70:
        return -2
    else:
        return 1



env = gym.make('simglucose-adolescent2-v0')

reward = 1
done = False

observation = env.reset()
for t in range(200):
    env.render(mode='human')
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print(observation)
    print("Reward = {}".format(reward))
    if done:
        print("Episode finished after {} timesteps".format(t + 1))
        break